In [17]:
import sys
from pathlib import Path
from glob import glob
from importlib import reload

from pydicom import dcmread
import matplotlib.pyplot as plt
import numpy as np

# Add repo root to Python path
sys.path.append(str(Path().resolve().parent))

In [18]:
from src.config import SERIES_DIR, TRAIN_CSV
from src import dataloader as dl
from src import partition as pt

In [22]:
reload(dl)
reload(pt)

<module 'src.partition' from '/home/aaron/aneurisym-detection/src/partition.py'>

In [23]:
labels = dl.build_labels_map_from_csv(TRAIN_CSV)

dl_thin, dl_thick, ds_thin, ds_thick = pt.make_loaders(
    SERIES_DIR,
    labels_map=labels,
    z_thresh_mm=1.5,               # thin vs thick cutoff
    thin_out=(1.0,1.0,1.0),        # isotropic for thin stacks
    thick_xy=1.0,                  # keep Z native for thick stacks
    thin_patch=(96,192,192),
    thick_patch=(16,192,192),
    batch_size_thin=1,
    batch_size_thick=1,
    num_workers=0,
    shuffle=True,
)

# 2) Quick sanity
b_thin  = next(iter(dl_thin))
b_thick = next(iter(dl_thick))
print("thin:",  tuple(b_thin["volume"].shape),  b_thin["spacing"])
print("thick:", tuple(b_thick["volume"].shape), b_thick["spacing"])

# Optional: overview of z-spacings
print(pt.scan_spacing_stats(SERIES_DIR)[:5])

thin: (1, 1, 96, 192, 192) [tensor([1.], dtype=torch.float64), tensor([1.], dtype=torch.float64), tensor([1.], dtype=torch.float64)]
thick: (1, 1, 16, 192, 192) [tensor([18.], dtype=torch.float64), tensor([1.], dtype=torch.float64), tensor([1.], dtype=torch.float64)]
[('1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647', 18.0), ('1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085', 0.7), ('1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317', 0.6), ('1.2.826.0.1.3680043.8.498.10009383108068795488741533244914370182', 0.75), ('1.2.826.0.1.3680043.8.498.10012790035410518400400834395242853657', 0.55)]
